In [2]:
# 采集分析 AMZN 的股票新闻并计算每周向量情绪得分 {即：负面、中性、正面、复合

import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer

web_url = 'https://finviz.com/quote.ashx?t='

news_tables = {}
tickers = ['AMZN', 'GOOG', 'TSLA']

for tick in tickers:
    url = web_url + tick
    req = Request(url=url,headers={"User-Agent": "Chrome"}) 
    response = urlopen(req)    
    html = BeautifulSoup(response,"html.parser")
    news_table = html.find(id='news-table')
    news_tables[tick] = news_table

amazon = news_tables['AMZN']
amazon_tr = amazon.findAll('tr')


In [3]:

# for x, table_row in enumerate(amazon_tr):
#     a_text = table_row.a.text
#     td_text = table_row.td.text
#     print(a_text)
#     print(td_text)
#     if x == 5:
#         break

for x, table_row in enumerate(amazon_tr):
    if table_row.a is not None:
        a_text = table_row.a.text
        print(a_text)
    else:
        print("No anchor tag found")

    if table_row.td is not None:
        td_text = table_row.td.text
        print(td_text)
    else:
        print("No td tag found")

    if x == 10:
        break

Here's Why Amazon (AMZN) Gained But Lagged the Market Today

            Today 05:45PM
        
Rigetti, IonQ stocks plunge after Nvidia CEO says 'useful' quantum computing is decades away

            04:07PM
        
Panama Canal CEO Says Meeting Trumps Demands Will Lead to Chaos

            02:48PM
        
Amazon AWS To Invest $11 Billion In Georgia To Boost AI And Cloud Infrastructure: 'Expect To Create More Than 550 Well-Paying Jobs'

            02:00PM
        
Intel Partners with Amazon Cloud to Drive Next-Gen Smart Vehicle Tech, Wins Stellantis, Karma Auto As Clients

            01:28PM
        
No anchor tag found
01:16PM
Cantor Fitzgerald Raises Amazon.com, Inc. (AMZN)s Price Target to $270, Highlighting Strong AI Growth and Future Potential

            01:16PM
        
"Motley Fool Money" 2025 Stock Market Preview

            12:55PM
        
Amazon (AMZN) Upgraded to Strong Buy: Here's Why

            12:00PM
        
Amazon's $11 Billion Investment Targets Cloud Gro

In [8]:
filtered_rows = [row for row in amazon_tr if row.a is not None and row.td is not None]

for x, table_row in enumerate(filtered_rows):
    # Extract and print <a> tag text
    a_text = table_row.a.text
    print(a_text)

    # Extract and print <td> tag text
    td_text = table_row.td.text
    print(td_text)

    if x == 10:
        break

Here's Why Amazon (AMZN) Gained But Lagged the Market Today

            Today 05:45PM
        
Rigetti, IonQ stocks plunge after Nvidia CEO says 'useful' quantum computing is decades away

            04:07PM
        
Panama Canal CEO Says Meeting Trumps Demands Will Lead to Chaos

            02:48PM
        
Amazon AWS To Invest $11 Billion In Georgia To Boost AI And Cloud Infrastructure: 'Expect To Create More Than 550 Well-Paying Jobs'

            02:00PM
        
Intel Partners with Amazon Cloud to Drive Next-Gen Smart Vehicle Tech, Wins Stellantis, Karma Auto As Clients

            01:28PM
        
Cantor Fitzgerald Raises Amazon.com, Inc. (AMZN)s Price Target to $270, Highlighting Strong AI Growth and Future Potential

            01:16PM
        
"Motley Fool Money" 2025 Stock Market Preview

            12:55PM
        
Amazon (AMZN) Upgraded to Strong Buy: Here's Why

            12:00PM
        
Amazon's $11 Billion Investment Targets Cloud Growth in Georgia

            

In [9]:
#  创建新闻列表

# news_list = []

# for file_name, news_table in news_tables.items():
#     for i in news_table.findAll('tr'):

#         text = i.a.get_text() 

#         date_scrape = i.td.text.split()

#         if len(date_scrape) == 1:
#             time = date_scrape[0]

#         else:
#             date = date_scrape[0]
#             time = date_scrape[1]

#         tick = file_name.split('_')[0]

#         news_list.append([tick, date, time, text])

# print(news_list)


news_list = []

for file_name, news_table in news_tables.items():

    for i in news_table.findAll('tr'):

        if i.a is not None:  # Check if i.a is not None

            text = i.a.get_text()

        else:

            text = "No title"  # Assign a default value if i.a is None

        date_scrape = i.td.text.split()

        if len(date_scrape) == 1:

            time = date_scrape[0]

        else:

            date = date_scrape[0]

            time = date_scrape[1]

        tick = file_name.split('_')[0]

        news_list.append([tick, date, time, text])

print(news_list)



[['AMZN', 'Today', '05:45PM', "Here's Why Amazon (AMZN) Gained But Lagged the Market Today"], ['AMZN', 'Today', '04:07PM', "Rigetti, IonQ stocks plunge after Nvidia CEO says 'useful' quantum computing is decades away"], ['AMZN', 'Today', '02:48PM', 'Panama Canal CEO Says Meeting Trumps Demands Will Lead to Chaos'], ['AMZN', 'Today', '02:00PM', "Amazon AWS To Invest $11 Billion In Georgia To Boost AI And Cloud Infrastructure: 'Expect To Create More Than 550 Well-Paying Jobs'"], ['AMZN', 'Today', '01:28PM', 'Intel Partners with Amazon Cloud to Drive Next-Gen Smart Vehicle Tech, Wins Stellantis, Karma Auto As Clients'], ['AMZN', 'Today', '01:16PM', 'No title'], ['AMZN', 'Today', '01:16PM', 'Cantor Fitzgerald Raises Amazon.com, Inc. (AMZN)s Price Target to $270, Highlighting Strong AI Growth and Future Potential'], ['AMZN', 'Today', '12:55PM', '"Motley Fool Money" 2025 Stock Market Preview'], ['AMZN', 'Today', '12:00PM', "Amazon (AMZN) Upgraded to Strong Buy: Here's Why"], ['AMZN', 'Today'

In [14]:
#  创建新闻列表

news_list = []

for file_name, news_table in news_tables.items():
    filtered_rows = [row for row in news_table.findAll('tr') if row.a is not None]

    for row in filtered_rows:
        # Extract the text from the <a> tag
        text = row.a.get_text()

        # Extract and split the <td> content
        date_scrape = row.td.text.split()

        # Handle cases for "Today", "Yesterday", and time-only entries
        if len(date_scrape) == 1:
            if date_scrape[0] in ["Today", "Yesterday"]:
                date = date_scrape[0]
                time = None  # No specific time in this case
            else:
                date = None
                time = date_scrape[0]  # Time-only entry
        else:
            date = date_scrape[0]  # Extract date
            time = date_scrape[1]  # Extract time

        # Extract the ticker symbol
        tick = file_name.split('_')[0]

        # Append the processed information to the news list
        news_list.append([tick, date, time, text])

#print(news_list)


for entry in news_list:
    print(f"Ticker: {entry[0]}, Date: {entry[1]}, Time: {entry[2]}, Title: {entry[3]}")
    

Ticker: AMZN, Date: Today, Time: 05:45PM, Title: Here's Why Amazon (AMZN) Gained But Lagged the Market Today
Ticker: AMZN, Date: None, Time: 04:07PM, Title: Rigetti, IonQ stocks plunge after Nvidia CEO says 'useful' quantum computing is decades away
Ticker: AMZN, Date: None, Time: 02:48PM, Title: Panama Canal CEO Says Meeting Trumps Demands Will Lead to Chaos
Ticker: AMZN, Date: None, Time: 02:00PM, Title: Amazon AWS To Invest $11 Billion In Georgia To Boost AI And Cloud Infrastructure: 'Expect To Create More Than 550 Well-Paying Jobs'
Ticker: AMZN, Date: None, Time: 01:28PM, Title: Intel Partners with Amazon Cloud to Drive Next-Gen Smart Vehicle Tech, Wins Stellantis, Karma Auto As Clients
Ticker: AMZN, Date: None, Time: 01:16PM, Title: Cantor Fitzgerald Raises Amazon.com, Inc. (AMZN)s Price Target to $270, Highlighting Strong AI Growth and Future Potential
Ticker: AMZN, Date: None, Time: 12:55PM, Title: "Motley Fool Money" 2025 Stock Market Preview
Ticker: AMZN, Date: None, Time: 12:

In [15]:
from prettytable import PrettyTable

# Create a PrettyTable instance
table = PrettyTable()
table.field_names = ["Ticker", "Date", "Time", "Title"]

# Add rows to the table
for entry in news_list:
    table.add_row(entry)

# Set alignment: Left align the "Title" column
table.align["Title"] = "l"

# Print the table
print(table)


+--------+-----------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Ticker |    Date   |   Time  | Title                                                                                                                                                              |
+--------+-----------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|  AMZN  |   Today   | 05:45PM | Here's Why Amazon (AMZN) Gained But Lagged the Market Today                                                                                                        |
|  AMZN  |    None   | 04:07PM | Rigetti, IonQ stocks plunge after Nvidia CEO says 'useful' quantum computing is decades away                                                                       |
|  AMZN  |